In [1]:
%load_ext dotenv
%dotenv

In [2]:
import logging
import os
from zeta_py.zeta_client.accounts.pricing import Pricing
from solana.rpc.async_api import AsyncClient
from zeta_py.client import Client
from zeta_py.types import Network, Asset, Side
import anchorpy
from solders.pubkey import Pubkey
from zeta_py import pda
from solana.rpc.commitment import Confirmed, Finalized
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType
from solana.rpc.types import TxOpts

logger = logging.getLogger(f"zeta_py")
logging.basicConfig(level=logging.DEBUG)

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])
conn = AsyncClient(endpoint, commitment=Confirmed, blockhash_cache=False)

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
print(wallet.public_key)

https://mainnet.helius-rpc
4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh


In [26]:
# print(f"Balance: {(await conn.get_balance(wallet.public_key)).value / 1e9} SOL")

Balance: 0.21923837 SOL


In [3]:
tx_opts = TxOpts({"preflight_commitment": Confirmed, "skip_confirmation": False})

client = await Client.load(Network.MAINNET, conn, wallet, [Asset.SOL], subscribe=True, tx_opts=tx_opts)

INFO:zeta_py.accounts.Account:Loaded account: State
INFO:zeta_py.accounts.Account:Loaded account: Pricing
INFO:zeta_py.market.Market.SOL:Subscribed to SOL:bid
INFO:zeta_py.market.Market.SOL:Subscribed to SOL:ask
INFO:zeta_py.accounts.Account:Loaded account: Clock
INFO:zeta_py.accounts.Account:Subscribed to Clock
INFO:zeta_py.accounts.Account:Subscribed to Pricing
INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccount


In [28]:
# print(client.balance)
# print(client.positions)
# print(client.open_orders)

0.00904
{<Asset.SOL: 0>: Position(size=-0.004, cost_of_trades=0.082259)}
{<Asset.SOL: 0>: []}


In [29]:
# await client.deposit(0.000001)

INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccountManager
INFO:zeta_py.client.Client:Deposit of $1e-06 USDC to margin account Hu2yXbPESXyamLdvDemeASCFd3rHnpN2xBDUJpnuG5UX submitted


Signature(
    34moN2zwVg9PvrtGCX9iwWW84D3amuvQftNNVRvp7qBx5DZMchCR1ZBzZBRi9SKCVtrzi8vkHYyiWzWNjTZhXcih,
)

In [7]:
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType

expiry_ts = int((datetime.now()+timedelta(seconds=1000)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.PostOnlySlide, client_order_id=69)
await client.place_order(Asset.SOL, 0.1, 0.001, Side.Ask, order_opts)

INFO:zeta_py.client.Client:Placed 0.001x SOL-PERP Ask @ $0.1


Signature(
    2pTfh4ZU4y7v5NQZLacn7HUdPWVo9muDGiXCG4nZUhuPWGHnrgXsZ3T2ePznuTZNqeXZsaigW88ZpmjJBoHmQYu8,
)

In [15]:
open_orders = await client.fetch_open_orders(Asset.SOL)
open_orders

[]

In [14]:
await client.cancel_order(Asset.SOL, order_id=open_orders[0].order_id, side=Side.Ask)

RPCException: SendTransactionPreflightFailureMessage { message: "Transaction simulation failed: Error processing Instruction 0: custom program error: 0x3b", data: RpcSimulateTransactionResult(RpcSimulateTransactionResult { err: Some(InstructionError(0, Custom(59))), logs: Some(["Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD invoke [1]", "Program log: Instruction: CancelOrder", "Program zDEXqXEG7gAyxb1Kg9mK5fPnUdENCGKzWrM21RMdWRq invoke [2]", "Program zDEXqXEG7gAyxb1Kg9mK5fPnUdENCGKzWrM21RMdWRq consumed 3525 of 182516 compute units", "Program zDEXqXEG7gAyxb1Kg9mK5fPnUdENCGKzWrM21RMdWRq failed: custom program error: 0x3b", "Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD consumed 21009 of 200000 compute units", "Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD failed: custom program error: 0x3b"]), accounts: None, units_consumed: Some(0), return_data: None }) }

In [8]:
from solana.transaction import Transaction
from zeta_py import constants, utils
from zeta_py.types import OrderOptions
from spl.token.constants import ASSOCIATED_TOKEN_PROGRAM_ID, TOKEN_PROGRAM_ID
from solders.sysvar import RENT


from zeta_py.zeta_client.instructions import (
    deposit_v2,
    initialize_cross_margin_account,
    initialize_cross_margin_account_manager,
    place_perp_order_v3,
)

tx = Transaction()

asset = Asset.SOL
price = 0.1
size=0.001
side = Side.Bid
order_opts = OrderOptions

tx.add(
    place_perp_order_v3(
                {
                    "price": utils.convert_decimal_to_fixed_int(price),
                    "size": utils.convert_decimal_to_fixed_lot(size),
                    "side": side.to_program_type(),
                    "order_type": order_opts.order_type.to_program_type(),
                    "client_order_id": order_opts.client_order_id,
                    "tif_offset": None,  # TODO: this later `getTIFOffset`
                    "tag": order_opts.tag,
                    "asset": asset.to_program_type(),
                },
                {
                    "state": client.exchange.state.address,
                    "pricing": client.exchange.pricing.address,
                    "margin_account": client.margin_account.address,
                    "authority": client.provider.wallet.public_key,
                    "dex_program": constants.DEX_PID[client.network],
                    "token_program": TOKEN_PROGRAM_ID,
                    "serum_authority": client.exchange._serum_authority_address,
                    "open_orders": client._open_orders_addresses[asset],
                    "rent": RENT,
                    "market_accounts": {
                        "market": client.exchange.markets[asset].address,
                        "request_queue": client.exchange.markets[asset]._serum_market.state.request_queue(),
                        "event_queue": client.exchange.markets[asset]._serum_market.state.event_queue(),
                        "bids": client.exchange.markets[asset]._serum_market.state.bids(),
                        "asks": client.exchange.markets[asset]._serum_market.state.asks(),
                        "order_payer_token_account": client.exchange.markets[asset]._quote_zeta_vault_address
                        if side == Side.Bid
                        else client.exchange.markets[asset]._base_zeta_vault_address,
                        "coin_vault": client.exchange.markets[asset]._serum_market.state.base_vault(),
                        "pc_vault": client.exchange.markets[asset]._serum_market.state.quote_vault(),
                        "coin_wallet": client.exchange.markets[asset]._base_zeta_vault_address,
                        "pc_wallet": client.exchange.markets[asset]._quote_zeta_vault_address,
                    },
                    "oracle": client.exchange.pricing.account.oracles[asset.to_index()],
                    "oracle_backup_feed": client.exchange.pricing.account.oracle_backup_feeds[asset.to_index()],
                    "oracle_backup_program": constants.CHAINLINK_PID,
                    "market_mint": client.exchange.markets[asset]._serum_market.state.quote_mint()
                    if side == Side.Bid
                    else client.exchange.markets[asset]._serum_market.state.base_mint(),
                    "mint_authority": client.exchange._mint_authority_address,
                    "perp_sync_queue": client.exchange.pricing.account.perp_sync_queues[asset.to_index()],
                },
            )
)

# TODO: grab recent blockhash??
signed_tx = client.provider.wallet.sign_transaction(tx)
signed_tx.verify_signatures()

await client.provider.simulate(signed_tx)

InvalidParamsMessage {
    message: "invalid transaction: Transaction failed to sanitize accounts offsets correctly",
}